In [1]:
import pandas as pd
import json
import yaml

In [2]:
oso = pd.read_csv("RetroPGF3 Projects - OSO Snapshot.csv", index_col=0)
slug_mapping = dict(oso['OSO Slug'])

In [3]:
df = pd.read_csv("RetroPGF3 Results - ConsolidatedData.csv")
df['slug'] = df['Project ID'].map(slug_mapping)
url = "https://vote.optimism.io/retropgf/3/application/"
df['link'] = df['Project ID'].apply(lambda x: f"{url}{x}")
df.rename(columns={'Meta: Project Name': 'project', 'Result: Received OP': 'amount'}, inplace=True)
df = df[['project', 'link', 'amount', 'slug']]
df['tags'] = "RetroPGF3"

df.loc[122,'slug'] = 'rainbow'
df.loc[33,'slug'] = 'gitcoin'

df.head()

,project,link,amount,slug,tags
0,Protocol Guild,https://vote.optimism.io/retropgf/3/applicatio...,663853.62,protocol-guild,RetroPGF3
1,go-ethereum,https://vote.optimism.io/retropgf/3/applicatio...,496896.42,go-ethereum,RetroPGF3
2,Solidity,https://vote.optimism.io/retropgf/3/applicatio...,422361.96,solidity,RetroPGF3
3,Erigon,https://vote.optimism.io/retropgf/3/applicatio...,339545.72,erigon-ledgerwatch,RetroPGF3
4,ethers.js,https://vote.optimism.io/retropgf/3/applicatio...,298137.85,ethers-io,RetroPGF3


In [4]:
df2 = pd.read_json("2023-10-01_op_tracked_grants.json")

def fix_tags(x):
    if isinstance(x, list):
        s = ", ".join(x)
    else:
        s = ", ".join(eval(x))
    return s.replace("RPGF", "RetroPGF")
        
df2['tags'] = df2['tags'].apply(fix_tags)

In [5]:
combined = pd.concat([df,df2], axis=0, ignore_index=True)
combined['slug'] = combined['slug'].apply(lambda x: x if x != "" else None)
combined

,project,link,amount,slug,tags
0,Protocol Guild,https://vote.optimism.io/retropgf/3/applicatio...,663853.62,protocol-guild,RetroPGF3
1,go-ethereum,https://vote.optimism.io/retropgf/3/applicatio...,496896.42,go-ethereum,RetroPGF3
2,Solidity,https://vote.optimism.io/retropgf/3/applicatio...,422361.96,solidity,RetroPGF3
3,Erigon,https://vote.optimism.io/retropgf/3/applicatio...,339545.72,erigon-ledgerwatch,RetroPGF3
4,ethers.js,https://vote.optimism.io/retropgf/3/applicatio...,298137.85,ethers-io,RetroPGF3
...,...,...,...,...,...
1073,Perpetuals Dune Abstraction,https://app.optimism.io/retropgf-discovery/0x0...,2069.00,None,RetroPGF2
1074,raise_exception (rotate.eth),https://app.optimism.io/retropgf-discovery/0x4...,2044.00,None,RetroPGF2
1075,Optimism Cult Merch,https://app.optimism.io/retropgf-discovery/0x5...,1863.00,None,RetroPGF2
1076,Thumbs Up Finance,https://app.optimism.io/retropgf-discovery/0x4...,725.00,None,RetroPGF2


In [6]:
d = yaml.safe_load(open("../dune/op_dune_namespaces_to_ossd.yaml"))
dune = pd.Series(d).reset_index()
dune.columns = ['dune_namespace', 'slug']
dune_mapping = dune.groupby('slug')['dune_namespace'].agg(lambda x: ", ".join(set(x)))

In [7]:
combined['dune_namespace'] = combined['slug'].map(dune_mapping)

In [8]:
combined.sort_values(by=['dune_namespace', 'slug', 'project']).reset_index(drop=True).to_csv("2024-02-06_op_tracked_grants.csv")

In [9]:
combined[combined['slug'].isna()==False]

,project,link,amount,slug,tags,dune_namespace
0,Protocol Guild,https://vote.optimism.io/retropgf/3/applicatio...,663853.62,protocol-guild,RetroPGF3,NaN
1,go-ethereum,https://vote.optimism.io/retropgf/3/applicatio...,496896.42,go-ethereum,RetroPGF3,NaN
2,Solidity,https://vote.optimism.io/retropgf/3/applicatio...,422361.96,solidity,RetroPGF3,NaN
3,Erigon,https://vote.optimism.io/retropgf/3/applicatio...,339545.72,erigon-ledgerwatch,RetroPGF3,NaN
4,ethers.js,https://vote.optimism.io/retropgf/3/applicatio...,298137.85,ethers-io,RetroPGF3,NaN
...,...,...,...,...,...,...
972,Gov,https://app.optimism.io/retropgf-discovery/0xa...,1080.00,gov-w3hc,RetroPGF2,NaN
973,ERC-5560: Redeemable NFTs,https://app.optimism.io/retropgf-discovery/0x5...,1073.00,ato-nft,RetroPGF2,NaN
974,Questbook,https://app.optimism.io/retropgf-discovery/0x4...,1015.00,questbook,RetroPGF2,NaN
975,Rigoblock,https://app.optimism.io/retropgf-discovery/0x0...,928.00,rigo-block,RetroPGF2,RigoBlock


In [10]:
combined['amount'].sum()

108792336.64